## Importing Libraries and Setting Up Environment

In [53]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle

import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

In [54]:
heart_disease_df = pd.read_csv('../data/heart_disease.csv')

## Exploratory Data Analysis (EDA)

In [55]:
heart_disease_df.shape

(920, 16)

In [56]:
heart_disease_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB


In [57]:
print(heart_disease_df.head())

   id  age     sex    dataset               cp  trestbps   chol    fbs  \
0   1   63    Male  Cleveland   typical angina     145.0  233.0   True   
1   2   67    Male  Cleveland     asymptomatic     160.0  286.0  False   
2   3   67    Male  Cleveland     asymptomatic     120.0  229.0  False   
3   4   37    Male  Cleveland      non-anginal     130.0  250.0  False   
4   5   41  Female  Cleveland  atypical angina     130.0  204.0  False   

          restecg  thalch  exang  oldpeak        slope   ca  \
0  lv hypertrophy   150.0  False      2.3  downsloping  0.0   
1  lv hypertrophy   108.0   True      1.5         flat  3.0   
2  lv hypertrophy   129.0   True      2.6         flat  2.0   
3          normal   187.0  False      3.5  downsloping  0.0   
4  lv hypertrophy   172.0  False      1.4    upsloping  0.0   

                thal  num  
0       fixed defect    0  
1             normal    2  
2  reversable defect    1  
3             normal    0  
4             normal    0  


In [58]:
heart_disease_df.describe().round(2)

,id,age,trestbps,chol,thalch,oldpeak,ca,num
count,920.00,920.00,861.00,890.00,865.00,858.00,309.00,920.00
mean,460.50,53.51,132.13,199.13,137.55,0.88,0.68,1.00
std,265.73,9.42,19.07,110.78,25.93,1.09,0.94,1.14
min,1.00,28.00,0.00,0.00,60.00,-2.60,0.00,0.00
25%,230.75,47.00,120.00,175.00,120.00,0.00,0.00,0.00
50%,460.50,54.00,130.00,223.00,140.00,0.50,0.00,1.00
75%,690.25,60.00,140.00,268.00,157.00,1.50,1.00,2.00
max,920.00,77.00,200.00,603.00,202.00,6.20,3.00,4.00


## Data Quality Checks

In [59]:
heart_disease_df.isna().sum()

id            0
age           0
sex           0
dataset       0
cp            0
trestbps     59
chol         30
fbs          90
restecg       2
thalch       55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
num           0
dtype: int64

In [60]:
heart_disease_df.drop(heart_disease_df[['id', 'dataset']], axis=1, inplace=True)
heart_disease_df['num'] = heart_disease_df['num'].apply(lambda x: 1 if x > 0 else 0)

In [61]:
heart_disease_df.drop(heart_disease_df[heart_disease_df['thal'].isnull() & heart_disease_df['slope'].isnull()].index, inplace=True)

mean_cols = ['trestbps', 'chol', 'thalch', 'oldpeak']
heart_disease_df[mean_cols] = heart_disease_df[mean_cols].fillna(heart_disease_df[mean_cols].mean())

heart_disease_df['ca'].fillna(heart_disease_df['ca'].median(), inplace=True)

mode_cols = ['fbs', 'restecg', 'exang', 'slope', 'thal']
for col in mode_cols:
    heart_disease_df[col].fillna(heart_disease_df[col].mode().iloc[0], inplace=True)

In [62]:
heart_disease_df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalch      0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
num         0
dtype: int64

In [63]:
outlier_clos = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
outlier = pd.Series(False, index=heart_disease_df.index)

for col in outlier_clos:
    Q1 = heart_disease_df[col].quantile(0.25)
    Q3 = heart_disease_df[col].quantile(0.75)
    IQR = Q3 - Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR
    mask = ((heart_disease_df[col] > upper) | (heart_disease_df[col] < lower))
    outlier |= mask

heart_disease_df = heart_disease_df[~outlier]

## Data Preprocessing

### Encoding

In [64]:
heart_disease_df = pd.get_dummies(heart_disease_df, columns=['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal'], dtype=int)

columns = heart_disease_df.columns
with open('../.pkl/columns.pkl', 'wb') as f:
    pickle.dump(columns, f)

### Scaling

In [65]:
heart_disease_df_scaled = heart_disease_df.copy()

scaler = StandardScaler()
scale_clos = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
heart_disease_df_scaled[scale_clos] = scaler.fit_transform(heart_disease_df_scaled[scale_clos])

with open('../.pkl/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

## Save preprocessed dataset

In [66]:
heart_disease_df_scaled.to_csv('../data/heart_disease_preprocessed.csv', index=False)